In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models
import datetime
import time

# 加载数据集并进行预处理
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
data_path = '../data-unversioned/p1ch7/'
trainset = datasets.CIFAR10(root=data_path, train=True,
                            download=True, transform=transform_train)
testset = datasets.CIFAR10(root=data_path, train=False,
                           download=True, transform=transform_test)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=4)


Files already downloaded and verified
Files already downloaded and verified


In [9]:
import torch
# import torch.nn as nn
# import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# 定义残差块
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

# 定义残差网络
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self.make_layer(block, 64, layers[2], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

# 定义超参数
batch_size = 128
epochs = 20
lr = 0.1
momentum = 0.9
weight_decay = 1e-4
device = 'cpu'

# 加载数据集
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = datasets.CIFAR10(root=data_path, train=True, download=True, transform=train_transform)
test_dataset = datasets.CIFAR10(root=data_path, train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [10]:

#定义模型、损失函数和优化器
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

#训练模型
for epoch in range(epochs):
    train_loss = 0.0
    train_acc = 0.0
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        train_acc += torch.sum(preds == labels.data)

    train_loss /= len(train_loader.dataset)
    train_acc /= len(train_loader.dataset)
    print('Epoch: {}, Train Loss: {:.4f}, Train Acc: {:.4f}'.format(epoch+1, train_loss, train_acc))



Epoch: 1, Train Loss: 1.5880, Train Acc: 0.3998
Epoch: 2, Train Loss: 1.0727, Train Acc: 0.6118
Epoch: 3, Train Loss: 0.8569, Train Acc: 0.6957
Epoch: 4, Train Loss: 0.7255, Train Acc: 0.7465
Epoch: 5, Train Loss: 0.6665, Train Acc: 0.7692
Epoch: 6, Train Loss: 0.6055, Train Acc: 0.7897
Epoch: 7, Train Loss: 0.5705, Train Acc: 0.8036
Epoch: 8, Train Loss: 0.5418, Train Acc: 0.8116
Epoch: 9, Train Loss: 0.5130, Train Acc: 0.8231
Epoch: 10, Train Loss: 0.4962, Train Acc: 0.8301
Epoch: 11, Train Loss: 0.4774, Train Acc: 0.8352
Epoch: 12, Train Loss: 0.4652, Train Acc: 0.8379
Epoch: 13, Train Loss: 0.4499, Train Acc: 0.8437
Epoch: 14, Train Loss: 0.4414, Train Acc: 0.8485
Epoch: 15, Train Loss: 0.4212, Train Acc: 0.8539
Epoch: 16, Train Loss: 0.4185, Train Acc: 0.8565
Epoch: 17, Train Loss: 0.4072, Train Acc: 0.8607
Epoch: 18, Train Loss: 0.4040, Train Acc: 0.8597
Epoch: 19, Train Loss: 0.3927, Train Acc: 0.8637
Epoch: 20, Train Loss: 0.3863, Train Acc: 0.8645
Epoch: 20, Train Loss: 0.0000

In [11]:
# 在测试集上评估模型
model.eval()
test_loss = 0.0
test_acc = 0.0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        test_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        test_acc += torch.sum(preds == labels.data)

    test_loss /= len(test_loader.dataset)
    test_acc /= len(test_loader.dataset)


    train_loss /= len(train_loader.dataset)
    train_acc /= len(train_loader.dataset)

    print('Epoch: {}, Train Loss: {:.4f}, Train `Acc: {:.4f}'.format(epoch+1, train_loss, train_acc))


Epoch: 20, Train Loss: 0.0000, Train `Acc: 0.0000


In [12]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        with torch.no_grad():  # <1>
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())  # <4>
        print("Accuracy {}: {:.2f}".format(name , correct / total))

print(datetime.datetime.now(), '开始运算')
validate(model, train_loader, val_loader)
print(datetime.datetime.now(), '结束运算')

2023-06-03 20:19:41.522317 开始运算
Accuracy train: 0.86
Accuracy val: 0.84
2023-06-03 20:21:17.933863 结束运算
